# Convert DICOM to NIFTI

Gather necessary pre-requisites

In [1]:
# import everything you need
from nipype.interfaces.dcm2nii import Dcm2niix
from nipype import Node, Workflow
import os, re


In [4]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=[]
    for i in os.listdir(dir):
        # verify subject number in directory and exclude other dirs
        if os.path.isdir(os.path.join(dir, i))\
        and re.match(r'(.*\d{3}$)', i):
            subdirs.append(os.path.join(dir, i))
    return sorted(subdirs)

# get iterator to go over multiple objects at once
def grouped(iterable, n):
    "s -> (s0,s1,s2,...sn-1), (sn,sn+1,sn+2,...s2n-1), (s2n,s2n+1,s2n+2,...s3n-1), ..."
    return zip(*[iter(iterable)]*n)


In [6]:
# get subject dir-list
subdirs=get_subdirs("/fMRI/")
print(subdirs)
print(len(subdirs))

['/fMRI/FED006', '/fMRI/FED007', '/fMRI/FED008', '/fMRI/FED009', '/fMRI/FED010', '/fMRI/FED011', '/fMRI/FED012', '/fMRI/FED013', '/fMRI/FED014', '/fMRI/FED015', '/fMRI/FED016', '/fMRI/FED017', '/fMRI/FED018', '/fMRI/FED019', '/fMRI/FED020', '/fMRI/FED021', '/fMRI/FED022', '/fMRI/FED023', '/fMRI/FED024', '/fMRI/FED025', '/fMRI/FED026', '/fMRI/FED027', '/fMRI/FED028', '/fMRI/FED029', '/fMRI/FED030', '/fMRI/FED031', '/fMRI/FED032', '/fMRI/FED033', '/fMRI/FED034', '/fMRI/FED035', '/fMRI/FED036', '/fMRI/FED037', '/fMRI/FED038', '/fMRI/FED039', '/fMRI/FED040', '/fMRI/FED041', '/fMRI/FED042', '/fMRI/FED043', '/fMRI/FED044', '/fMRI/FED045', '/fMRI/FED046', '/fMRI/FED047', '/fMRI/FED048', '/fMRI/FED049', '/fMRI/FED050', '/fMRI/FED051', '/fMRI/FED052', '/fMRI/FED053', '/fMRI/FED054', '/fMRI/FED055', '/fMRI/FED056', '/fMRI/FED057', '/fMRI/FED058', '/fMRI/FED059', '/fMRI/FED060', '/fMRI/FED061', '/fMRI/FED062', '/fMRI/FED063', '/fMRI/FED064', '/fMRI/FED065', '/fMRI/FED066', '/fMRI/FED067', '/fMRI/

Select the required files

In [7]:
# get fMRI sequence dirs via subject lists
FMRIdirs=[]
for i in subdirs:
    for j in os.listdir(i):
        if os.path.isdir(os.path.join(i, j))\
        and re.match(r'((WIP840|FMRI).*\d{4}$)', j):
            FMRIdirs.append(os.path.join(i,j))
                
# same for gradient field maps
GREfielddirs=[]
for i in subdirs:
    for j in os.listdir(i):
        if os.path.isdir(os.path.join(i, j))\
        and re.match(r'((GRE_FIELD).*\d{4}$)', j):
            GREfielddirs.append(os.path.join(i,j))       
                
# sort the list entries
FMRIdirs=sorted(FMRIdirs)
GREfielddirs=sorted(GREfielddirs)

# extract the middle number of however many fMRI sequence dirs there are to select the functional runs (VARIATIONS. SOLVE.)
funcdirs=[]
for i in subdirs:
    dirs=[]
    for j in FMRIdirs:
        if i.rsplit('/', 1)[1] == j.rsplit('/', 2)[1]:
            dirs.append(j)
    funcdirs.append(sorted(dirs)[len(dirs) // 2])
    
# think of pattern for selecting the "right maps...those after the fMRI...likely^^"
distcordirs=[]


Make sure it all looks right

In [8]:
print(FMRIdirs)
print(len(FMRIdirs))

['/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0007', '/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009', '/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0011', '/fMRI/FED007/WIP840_36SLICE_1800TR_FS_0006', '/fMRI/FED007/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED007/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED008/WIP840_36SLICE_1800TR_FS_0006', '/fMRI/FED008/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED008/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED009/WIP840_36SLICE_1800TR_FS_0006', '/fMRI/FED009/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED009/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED010/WIP840_36SLICE_1800TR_FS_0006', '/fMRI/FED010/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED011/WIP840_36SLICE_1800TR_FS_0006', '/fMRI/FED011/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED011/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED012/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED012/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED012/WIP840_36SLICE_1800TR_FS_0012', '/fMRI/FED013/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED013/WIP840_36SLICE_1800

In [10]:
print(GREfielddirs)
print(len(GREfielddirs))

['/fMRI/FED006/GRE_FIELD_MAPPING_0006', '/fMRI/FED007/GRE_FIELD_MAPPING_0012', '/fMRI/FED007/GRE_FIELD_MAPPING_0013', '/fMRI/FED008/GRE_FIELD_MAPPING_0012', '/fMRI/FED008/GRE_FIELD_MAPPING_0020', '/fMRI/FED009/GRE_FIELD_MAPPING_0012', '/fMRI/FED009/GRE_FIELD_MAPPING_0020', '/fMRI/FED009/GRE_FIELD_MAPPING_0021', '/fMRI/FED010/GRE_FIELD_MAPPING_0010', '/fMRI/FED010/GRE_FIELD_MAPPING_0011', '/fMRI/FED011/GRE_FIELD_MAPPING_0016', '/fMRI/FED011/GRE_FIELD_MAPPING_0017', '/fMRI/FED012/GRE_FIELD_MAPPING_0006', '/fMRI/FED012/GRE_FIELD_MAPPING_0007', '/fMRI/FED013/GRE_FIELD_MAPPING_0006', '/fMRI/FED013/GRE_FIELD_MAPPING_0007', '/fMRI/FED014/GRE_FIELD_MAPPING_0010', '/fMRI/FED014/GRE_FIELD_MAPPING_0011', '/fMRI/FED015/GRE_FIELD_MAPPING_0006', '/fMRI/FED015/GRE_FIELD_MAPPING_0007', '/fMRI/FED016/GRE_FIELD_MAPPING_0006', '/fMRI/FED016/GRE_FIELD_MAPPING_0007', '/fMRI/FED017/GRE_FIELD_MAPPING_0012', '/fMRI/FED017/GRE_FIELD_MAPPING_0013', '/fMRI/FED018/GRE_FIELD_MAPPING_0006', '/fMRI/FED018/GRE_FIELD_

In [9]:
print(funcdirs)
print(len(funcdirs))

['/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009', '/fMRI/FED007/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED008/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED009/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED010/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED011/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED012/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED013/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED014/WIP840_36SLICE_1800TR_FS_0018', '/fMRI/FED015/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED016/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED017/WIP840_36SLICE_1800TR_FS_0008', '/fMRI/FED018/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED019/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED020/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED021/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED022/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED023/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED024/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED025/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED026/WIP840_36SLICE_1800TR_FS_0010', '/fMRI/FED027/WIP840_36SLICE_1800

In [11]:
print(distcordirs)
print(len(distcordirs))

[]
0


Convert .IMA files to NIFTI

In [12]:
# use sequence-list as source_dir and save output of converter in parent dir

# iterate over sequence and subject dirs, create nodes and run them to create NIFTI files
for i,j in zip(funcdirs, subdirs):
    print(i,j)
    # create DICOM-to-NIFTI converter node
    converter=Node(Dcm2niix(), name='converter')
    converter.inputs.compress='n' # do not compress -> SPM options etc.
    converter.inputs.source_dir=i
    converter.inputs.output_dir=j
    converter.run()

/fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009 /fMRI/FED006
200902-13:55:20,646 nipype.workflow INFO:
	 [Node] Setting-up "converter" in "/tmp/tmptdxewzd8/converter".
200902-13:55:20,651 nipype.workflow INFO:
	 [Node] Running "converter" ("nipype.interfaces.dcm2nii.Dcm2niix"), a CommandLine Interface with command:
dcm2niix -b y -z n -x n -t n -m n -o /fMRI/FED006 -s n -v n /fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009
200902-13:55:21,791 nipype.interface INFO:
	 stdout 2020-09-02T13:55:21.790871:Chris Rorden's dcm2niiX version v1.0.20200427  (JP2:OpenJPEG) GCC6.3.0 (64-bit Linux)
200902-13:55:21,792 nipype.interface INFO:
	 stdout 2020-09-02T13:55:21.790871:Found 280 DICOM file(s)
200902-13:55:21,793 nipype.interface INFO:
	 stdout 2020-09-02T13:55:21.790871:Convert 280 DICOM as /fMRI/FED006/WIP840_36SLICE_1800TR_FS_0009_WIP840_36slice_1800TR_fs_20141106153429_9 (72x68x36x280)
200902-13:55:21,867 nipype.interface INFO:
	 stdout 2020-09-02T13:55:21.867606:Conversion required 1.074133 seconds (

Script ends here